# Predykcja z użyciem kolumny `RESIGN_DATE`

## Importy bibliotek

In [ ]:
from datetime import datetime

import pandas as pd
import numpy as np

import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.cluster import KMeans
import xgboost as xgb
from kneed import KneeLocator

## Zczytanie danych z plików csv

* Pliki znajdują sie w folderze data i mają nazwy
    * PPK_Uczestnicy.csv
    * PPK_Pracodawcy.csv
* Po zczytaniu operacja merge złaczamy dane tak, że informacja o pracodawcy jest dostepna z poziomu pracownika

In [ ]:
uczestnicy = pd.read_csv('data/PPK_Uczestnicy.csv', sep=';')
pracodawcy = pd.read_csv('data/PPK_Pracodawcy.csv',sep=';')

df = pd.merge(
    uczestnicy,
    pracodawcy,
    how='left',
    left_on='EMPL_ID',
    right_on='ID'
)

## Zmiana kolumnn i początkowe wyczyszczenie DataFrame'u 



* Kolumna Age ta niestety została obarczona przenikami przez co była błędnie reprezentowana jako typ String, przerabiamy ją na typ int

* Drop'owanie niepotrzebnych kolumn które nie będą przydatne

* Przerobienie narodowości na mniejszą ilość podgrup, decyzja o przejsciu na wszystkie grupy z iloscia osob mniejsza niz 5000 naleza do jednej narodowsci

* Zmiana kolumn kategorycznych na dummy variables - tworzymy nową kolumnę na każdy możliwy output kolumny, porównaj poprzednią i następną komórkę

* Znajdowanie `DURATION`, które jest różnicą między data rezygnacji (`RESIGN_DATE`), a data podpisanie (`SIGN_DATE`), typ to int - liczba dni
    * ustawiamy także kolumne resigned (bit oznaczający czy ktoś zrezygnował oznaczane jako istnienie `RESIGN_DATE`)

In [ ]:
def to_int(age):
    age = age[:age.find(",")]
    return int(age)

df["AGE"] = df["AGE"].apply(to_int)

In [ ]:
df.drop(columns=['MEMBER_ID','EMPL_ID', 'WORK_START', 'WORK_STOP', 'LOGICAL_FACTOR_1','LOGICAL_FACTOR_2','ID','PKD_CODE','PPK_BANK','NUMERICAL_VALUE'], inplace=True)
df.drop(columns=["CREATED_AT", "UOZ_START_DATE", "UOP_SIGN_DATE", 'REGION_CODE'], inplace=True)

In [ ]:
# Grouping other nationalities into one category '0'
unique_nat = df['NATIONALITY'].unique()
for val in unique_nat:
    suma = ( df['NATIONALITY'].values == val ).sum()
    if suma < 5000:
        df.loc[df.NATIONALITY == val, 'NATIONALITY'] = 0

In [ ]:
df = pd.get_dummies(df, columns = ['SEX', 'COMPANY_SIZE', 'COMPANY_TYPE', 'VOIVODESHIP', 'NATIONALITY','PPK_STAGE'], 
                         prefix = ['SEX', 'COMPANY_SIZE', 'COMPANY_TYPE', 'VOIVODESHIP', 'NATIONALITY','PPK_STAGE'])

In [ ]:
def find_period(data):
    start, stop = data
    if type(stop) is float:
        stop = datetime.now().strftime("%Y-%m-%d")
    
    start_year, start_month, start_day = map(int, start.split('-'))
    stop_year, stop_month, stop_day = map(int, stop.split('-'))
    
    days = (stop_year - start_year)*365
    days += (stop_month - start_month)*30 if stop_month > start_month else (start_month - stop_month)*30
    days += stop_day - start_day if stop_day > start_day else start_day - stop_day
    
    return days


df['DURATION'] = list(zip(df['SIGN_DATE'], df['RESIGN_DATE']))

df['DURATION'] = df['DURATION'].apply(find_period) 

df['RESIGNED'] = ~df['RESIGN_DATE'].isna()

df.drop(columns = ['SIGN_DATE', 'RESIGN_DATE'], inplace=True)

## Wyrzucenie niepasujących danych oraz TEST/TRAIN split

In [ ]:
# ta kolumna to w ogole jest 100% skorelowana z targetem, także usuwamy
df.drop(columns=["RESIGNED"], inplace = True)

In [ ]:
df.columns


## Klasteryzacja wieku

Z korelacją `IS_SUSPENDED`, nie sam podzial wieku

Zaczynamy od sprawdzenia jaka ilosc klastrów najlepiej się sprawdzi

In [ ]:
dataset = df[['IS_SUSPENDED', 'AGE']]

scaled_dataset = dataset.values

kmeans_kwargs = {
        "init": "random",
        "n_init": 10,
        "max_iter": 300,
        "random_state": 42,
    }
   
# A list holds the SSE values for each k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(scaled_dataset)
    sse.append(kmeans.inertia_)

plt.style.use('fivethirtyeight')
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("n_of_clusters")
plt.ylabel("SSE")
plt.show()


kl = KneeLocator(range(1, 11), sse, curve="convex", direction="decreasing")
n_of_clusters = kl.elbow

kmeans = KMeans(
    init='random',
    n_clusters = n_of_clusters,
    n_init=50,
    max_iter=500,
    random_state=42
    )

kmeans.fit(scaled_dataset)

df['AGE_CL'] = kmeans.predict(df[["IS_SUSPENDED", "AGE"]])

df.drop(columns=["AGE"], inplace=True)

## Budowanie modelu

In [ ]:
#POSITIVE = SUSPENDED

X_train, X_test, y_train, y_test = train_test_split(df[df.columns[1:]], df["IS_SUSPENDED"], stratify=df["IS_SUSPENDED"], test_size=0.10, random_state=56)

#building the model
xgb_model = xgb.XGBClassifier(max_depth=5, learning_rate=0.08, objective= 'binary:logistic',n_jobs=-1).fit(X_train, y_train)

print('Accuracy of XGB classifier on training set: {:.2f}'
       .format(xgb_model.score(X_train, y_train)))
print('Accuracy of XGB classifier on test set: {:.2f}'
       .format(xgb_model.score(X_test[X_train.columns], y_test)))

In [ ]:
y_pred = xgb_model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
from xgboost import plot_importance
fig, ax = plt.subplots(figsize=(10,8))
plot_importance(xgb_model, ax=ax)

In [ ]:
df['proba'] = xgb_model.predict_proba(df[X_train.columns])[:,1]
df[['AGE','proba']].head(50)

## Dane testowe

In [ ]:
testowe = pd.read_csv('data/PPK_Uczestnicy_TEST.csv',sep=';')

In [ ]:
testowe.columns

In [ ]:
testowe["RESIGN_DATE"].value_counts()

In [ ]:
testowe["RESIGN_DATE"].isna().sum()